### Fair Rent

Ben Phillips
[portfolio website](https://benjaminphillips22.github.io).

The python script in this notebook can also be found [here](github...)

#### Problem and Motivation

When my housemates and I moved into our new home, we had to decide who would get which room and at what price. The rooms were different sizes and one had an ensuite, so it wouldn’t be fair to split the rent equally between us.

Here I will present a potential method for assigning and pricing the rooms.
The housemates individually (and privately) determine what they would value each room at, with these values adding up to the total rent. For example, if the monthly rent for the house is \$1000 a month, housemate 1 may value room 1 at \$200, room 2 at \$300 and room 3 at \$500. It is important not to think of these values as bids, but rather what the housemate would be happy to pay if they were assigned to that room.

The objective function for this problem is to maximise the minimum utility.

I define ‘Utility’ as the difference between what a housemate was willing to pay for the room assigned to them, and what they actually have to pay. For our example with housemate 1, if they are assigned to room 1 with rent being \$150 per month, that would have a utility of \$50.
This objective function has several desirable outcomes for allocating rooms and determining prices. It assigns rooms to maximise the sum of what each housemate valued their assigned room at. This can be thought of as getting the most ‘value’ out of the house. Once the rooms are assigned, the sum of what each housemate valued their assigned room at will be larger than the total rent (unless all housemates had identical preferences). This means that each housemate’s rent needs to be reduced by a certain amount, what I have defined as utility. By maximising the minimum utility, we can determine the fairest way to divide the rent.

Lastly, in addition to the objective, there is a constraint that makes this solution ‘Envy Free’. Room and rent are ‘envy free’ if no housemate would wish to swap rooms with anyone else. Going back to our example with housemate 1, if they were assigned room 1 at \$150 and housemate 2 was assigned room 3 at \$400, then housemate 1 would want to swap. Housemate 1 would have a utility of \$50 for their assigned room but could have a utility of \$100 (\$400 is \$100 less than what housemate 1 values room 3 at) if they swapped with housemate 2. This would not be an ‘envy free’ solution.

The algorithm used here was first described in this paper, [Which Is the Fairest (Rent Division) of Them All?](http://procaccia.info/papers/rent.pdf)

Install packages and initialise the solver

In [1]:
from ortools.linear_solver import pywraplp
from itertools import permutations

solver = pywraplp.Solver('SolveIntegerProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)


Create the Decision Varibles

In [2]:

DV = {}

# The rent that each housemate will pay for the room assigned to them.
for h in range(1, 4):
    name = 'H'+str(h)
    DV[name] = solver.NumVar(0.0, solver.infinity(), name)

# Assigned Rooms
# These values will be 0 or 1 depending on which room a housemate is assigned to. For example, if
# housemate 1 is assigned to room 1, then H1_R1 will equal 1 and H1_R2 and H1_R3 will equal 0.
for h in range(1, 4):
    for r in range(1, 4):
        name = 'H' + str(h) + '_R' + str(r)
        DV[name] = solver.IntVar(0.0, 1.0, name)
        
# Define house mate utilities for convenience
for i in range(1, 4):
    for j in range(1, 4):
        name = 'H' + str(i) + '_swap_H' + str(j) + '_utility'
        DV[name] = solver.NumVar(-solver.infinity(), solver.infinity(), name)

# Minimum Utility, the minimum of the three house mate utilities.
DV['min_stay_utility'] = solver.NumVar(-solver.infinity(), solver.infinity(), 'min__stay_utility')


Enter the house mate preferences

In [3]:
H1_preferences = [35, 50, 65]
H2_preferences = [45, 52, 53]
H3_preferences = [30, 45, 75]

# check prefences all sum to the same amount
assert sum(H1_preferences) == sum(H2_preferences) == sum(H3_preferences)

preferences = {}
H_list = [H1_preferences, H2_preferences, H3_preferences]

for index, h in enumerate(H_list):
    for r in range(0, 3):
        name = 'H' + str(index+1) + '_R' + str(r+1)
        preferences[name] = h[r]


Create the constraints

For convenience, H1_utility is written as H1_swap_H1_utility

In [4]:
# The rents will sum to the total rent
# H1 + H2 + H3 == 150
c0 = solver.Constraint(sum(H1_preferences), sum(H1_preferences))
for h in range(1, 4):
    name = 'H' + str(h)
    c0.SetCoefficient(DV[name], 1)

# One room per housemate
# H1_R1 + H1_R2 + H1_R3 == 1
# H2_R1 + H2_R2 + H2_R3 == 1
# H3_R1 + H3_R2 + H3_R3 == 1
for h in range(1, 4):
    c = solver.Constraint(1, 1)
    for r in range(1, 4):
        c.SetCoefficient(DV['H' + str(h) + '_R' + str(r)], 1)

# One housemate per room
# H1_R1 + H2_R1 + H3_R1 == 1
# H1_R2 + H2_R2 + H3_R2 == 1
# H1_R3 + H2_R3 + H3_R3 == 1
for r in range(1, 4):
    c = solver.Constraint(1, 1)
    for h in range(1, 4):
        c.SetCoefficient(DV['H' + str(h) + '_R' + str(r)], 1)

# Utility
# H1_utility = H1_R1*35 + H1_R2*50 + H1_R3*65 – H1
# H2_utility = H2_R1*45 + H2_R2*52 + H2_R3*53 – H2
# H3_utility = H3_R1*30 + H3_R2*45 + H3_R3*75 – H3
# Utility of swapped rooms.
# H1_swap_H2_utility = H2_R1*35 + H2_R2*50 + H2_R3*65 – H2
# H1_swap_H3_utility = H3_R1*35 + H3_R2*50 + H3_R3*65 – H3
# H2_swap_H1_utility = H1_R1*45 + H1_R2*52 + H1_R3*53 – H1
# H2_swap_H3_utility = H3_R1*45 + H3_R2*52 + H3_R3*53 – H3
# H3_swap_H1_utility = H1_R1*30 + H1_R2*45 + H1_R3*75 – H1
# H3_swap_H2_utility = H2_R1*30 + H2_R2*45 + H2_R3*75 – H2
for h_i in range(1, 4):
    for h_j in range(1, 4):
        c = solver.Constraint(0.0, 0.0)
        name = 'H' + str(h_i) + '_swap_H' + str(h_j) + '_utility'
        c.SetCoefficient(DV[name], -1)
        c.SetCoefficient(DV['H' + str(h_j)], -1)
        for r in range(1, 4):
            c.SetCoefficient(DV['H' + str(h_j) + '_R' + str(r)], preferences['H' + str(h_i) + '_R' + str(r)])
        
# H1_swap_H2_utility, H1_swap_H3_utility <= H1_utility
# H2_swap_H1_utility, H2_swap_H3_utility <= H2_utility
# H3_swap_H1_utility, H3_swap_H2_utility <= H3_utility
perm = permutations([1, 2, 3], 2) 
for h_i, h_j in perm:
    # print(a, ' ', b)
    c = solver.Constraint(-solver.infinity(), 0.0)
    c.SetCoefficient(DV['H' + str(h_i) + '_swap_H' + str(h_i) + '_utility'], -1)
    c.SetCoefficient(DV['H' + str(h_i) + '_swap_H' + str(h_j) + '_utility'], 1)

# Make the min_stay_utility smaller or equal to the house mate
# utilities for staying
for h in range(1, 4):
    c = solver.Constraint(-solver.infinity(), 0.0)
    c.SetCoefficient(DV['min_stay_utility'], 1)
    c.SetCoefficient(DV['H' + str(h) + '_swap_H' + str(h) + '_utility'], -1)


Create the objective function

In [5]:
objective = solver.Objective()
objective.SetCoefficient(DV['min_stay_utility'], 1)

objective.SetMaximization()


Solve the problem and print the solution.

In [6]:

result_status = solver.Solve()
# The problem has an optimal solution.
assert result_status == pywraplp.Solver.OPTIMAL

# The solution looks legit (when using solvers other than
# GLOP_LINEAR_PROGRAMMING, verifying the solution is highly recommended!).
assert solver.VerifySolution(1e-7, True)

print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())

# The objective value of the solution.
print('Optimal objective value = %d' % solver.Objective().Value())
print()
# The value of each variable in the solution.


for key, value in DV.items():
    print(key, ' = ', round(value.solution_value()))


Number of variables = 22
Number of constraints = 25
Optimal objective value = 6

H1  =  44
H2  =  37
H3  =  69
H1_R1  =  0
H1_R2  =  1
H1_R3  =  0
H2_R1  =  1
H2_R2  =  0
H2_R3  =  0
H3_R1  =  0
H3_R2  =  0
H3_R3  =  1
H1_swap_H1_utility  =  6
H1_swap_H2_utility  =  -2
H1_swap_H3_utility  =  -4
H2_swap_H1_utility  =  8
H2_swap_H2_utility  =  8
H2_swap_H3_utility  =  -16
H3_swap_H1_utility  =  1
H3_swap_H2_utility  =  -7
H3_swap_H3_utility  =  6
min_stay_utility  =  6


You can see that house mates 1, 2 and 3 are assigned to rooms 2, 1 and 3, respectively. House mate one was willing to pay \$50 for room 2, and instead is paying only \$44, giving housemate 1 a utility of \$6 which is seen in the value of 'H1_swap_H1_utility'. This utility is higher than if housemate 1 swapped rooms with either of the two other housemates.

This algorithm produced \$6, \$8 and \$6 of utility for house mates 1, 2 and 3, respectively. 